In [ ]:
!pip install openai gdown PyMuPDF

In [ ]:
import os
import re
import numpy as np
import base64
import requests
from tqdm import tqdm
from PIL import Image
import io
import fitz
import pandas as pd
import random
import ast
from openai import OpenAI
import json
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom

In [ ]:
import os
from google.colab import userdata
from openai import OpenAI

# 환경 변수로 설정
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

## 1. PDF 페이지를 이미지로 저장

In [ ]:
# 이미지를 base64 형식으로 인코딩하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 이미지가 다단인지 확인하는 함수
def is_two_column(img_array, threshold=240, column_width=0.1):
    height, width = img_array.shape[:2]
    center_start = int(width * (0.5 - column_width/2))
    center_end = int(width * (0.5 + column_width/2))

    center_region = img_array[:, center_start:center_end]

    # 이미지가 컬러라면 RGB를 그레이스케일로 변환
    if len(img_array.shape) == 3:
        center_region = np.mean(center_region, axis=2)

    vertical_projection = np.mean(center_region, axis=1)
    white_ratio = np.mean(vertical_projection > threshold)

    return white_ratio > 0.9  # 중심 부분이 90% 이상 흰색일 때 다단으로 간주

In [ ]:
# PNG 이미지를 최적화하는 함수
def optimize_png(img):
    img_buffer = io.BytesIO()
    img.save(img_buffer, format='PNG', optimize=True, compress_level=6)
    optimized_img = Image.open(img_buffer)
    return optimized_img

# PDF 파일을 처리하여 각 페이지를 이미지로 저장하는 함수
def process_pdf(pdf_path, output_folder, dpi=300):
    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    for page_num in range(len(doc)):
        page = doc[page_num]
        zoom = dpi / 72
        mat = fitz.Matrix(zoom, zoom)
        pix = page.get_pixmap(matrix=mat, alpha=False)

        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img_array = np.array(img)

        # 이미지 최적화 후 저장
        optimize_png(img).save(os.path.join(output_folder, f'page_{page_num+1}.png'))

    doc.close()

In [ ]:
pdf_path = '/content/HTP해석.pdf'
output_folder = '.'
process_pdf(pdf_path, output_folder)

In [ ]:
files = os.listdir()
print(files)

['.config', 'page_2.png', 'page_3.png', 'page_6.png', 'page_5.png', 'page_4.png', 'page_7.png', 'HTP해석.pdf', 'page_1.png', 'sample_data']


## 2. PDF 페이지를 xml로 저장

In [ ]:
def has_meaningful_content(text):
    """텍스트가 의미있는 내용을 포함하는지 확인"""
    if not text:
        return False

    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    # 특수문자 디코딩
    text = text.replace('&lt;', '<').replace('&gt;', '>').replace('&amp;', '&')
    # 공백, 특수문자 제거
    text = re.sub(r'[\s\t\n\r\f\v]+', '', text)
    # div, page 같은 일반적인 태그 관련 단어 제거
    text = re.sub(r'(div|page|style|width|height|pt)', '', text, flags=re.IGNORECASE)

    # 숫자와 단위 제거
    text = re.sub(r'\d+\.?\d*\s*(pt|px|em|rem|%)', '', text)

    # 남은 텍스트가 있는지 확인
    return bool(text.strip())

def extract_content_with_fallback(page):
    """여러 방법을 순차적으로 시도하여 의미있는 내용 추출"""
    content = None
    error_messages = []

    # 1. 기본 텍스트 추출 시도
    try:
        text = page.get_text("text")
        if text and has_meaningful_content(text):
            root = ET.Element("page")
            text_elem = ET.SubElement(root, "text")
            text_elem.text = text.strip()
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"기본 텍스트 추출 실패: {str(e)}")

    # 2. 블록 단위 추출 시도
    try:
        blocks = page.get_text("blocks")
        meaningful_blocks = []
        for block in blocks:
            if has_meaningful_content(block[4]):
                meaningful_blocks.append(block[4].strip())

        if meaningful_blocks:
            root = ET.Element("page")
            blocks_elem = ET.SubElement(root, "blocks")
            for block_text in meaningful_blocks:
                block_elem = ET.SubElement(blocks_elem, "block")
                block_elem.text = block_text
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"블록 추출 실패: {str(e)}")

    # 3. 단어 단위 추출 시도
    try:
        words = page.get_text("words")
        meaningful_words = []
        for word in words:
            if has_meaningful_content(word[4]):
                meaningful_words.append(word[4].strip())

        if meaningful_words:
            root = ET.Element("page")
            words_elem = ET.SubElement(root, "words")
            for word_text in meaningful_words:
                word_elem = ET.SubElement(words_elem, "word")
                word_elem.text = word_text
            return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"단어 추출 실패: {str(e)}")

    # 4. rawdict 시도
    try:
        raw_dict = page.get_text("rawdict")
        if "blocks" in raw_dict:
            meaningful_text = []
            for block in raw_dict["blocks"]:
                if "lines" in block:
                    for line in block["lines"]:
                        if "spans" in line:
                            for span in line["spans"]:
                                if has_meaningful_content(span.get("text", "")):
                                    meaningful_text.append(span["text"].strip())

            if meaningful_text:
                root = ET.Element("page")
                text_elem = ET.SubElement(root, "text")
                text_elem.text = " ".join(meaningful_text)
                return ET.tostring(root, encoding='utf-8').decode('utf-8'), None
    except Exception as e:
        error_messages.append(f"rawdict 추출 실패: {str(e)}")

    # 모든 방법이 실패했거나 의미있는 내용이 없는 경우 None 반환
    return None, "\n".join(error_messages)

def convert_pdf_to_xml(pdf_path, output_folder):
    """PDF를 XML로 변환하는 메인 함수"""
    if not os.path.exists(pdf_path):
        print(f"PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return

    os.makedirs(output_folder, exist_ok=True)

    print(f"PDF 파일 열기: {pdf_path}")
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        print(f"페이지 {page_num + 1}/{len(doc)} 처리 중...")
        page = doc[page_num]

        content, errors = extract_content_with_fallback(page)
        output_file = os.path.join(output_folder, f"page_{page_num+1:02d}.xml")

        if content:
            print(f"XML 파일 저장 중: {output_file}")
            with open(output_file, "w", encoding="utf-8") as f:
                f.write(content)
            print(f"페이지 {page_num + 1} 처리 완료")
        else:
            print(f"페이지 {page_num + 1}에서 추출된 의미있는 내용이 없습니다.")
            # 빈 내용일 경우 파일 생성하지 않음
            if os.path.exists(output_file):
                os.remove(output_file)

    doc.close()
    print("모든 페이지 처리 완료")

In [ ]:
pdf_path = '/content/HTP해석.pdf'
output_folder = '.'
convert_pdf_to_xml(pdf_path, output_folder)

PDF 파일 열기: /content/HTP해석.pdf
페이지 1/7 처리 중...
XML 파일 저장 중: ./page_01.xml
페이지 1 처리 완료
페이지 2/7 처리 중...
XML 파일 저장 중: ./page_02.xml
페이지 2 처리 완료
페이지 3/7 처리 중...
XML 파일 저장 중: ./page_03.xml
페이지 3 처리 완료
페이지 4/7 처리 중...
XML 파일 저장 중: ./page_04.xml
페이지 4 처리 완료
페이지 5/7 처리 중...
XML 파일 저장 중: ./page_05.xml
페이지 5 처리 완료
페이지 6/7 처리 중...
XML 파일 저장 중: ./page_06.xml
페이지 6 처리 완료
페이지 7/7 처리 중...
XML 파일 저장 중: ./page_07.xml
페이지 7 처리 완료
모든 페이지 처리 완료


In [ ]:
# 디렉토리에서 매칭되는 XML과 PNG 파일들을 가져오는 함수
def get_matched_files(directory):
    files = os.listdir(directory)

    xml_files = {}
    png_files = {}

    # XML과 PNG 파일들을 분류
    for file in files:
        match = re.match(r'page_(\d+)\.(xml|png)$', file)
        if match:
            number = int(match.group(1))
            extension = match.group(2)
            if extension == 'xml':
                xml_files[number] = file
            elif extension == 'png':
                png_files[number] = file

    # 모든 페이지 번호를 결합하여 매칭
    all_numbers = set(xml_files.keys()).union(set(png_files.keys()))

    matched_files = []
    for number in sorted(all_numbers):
        xml_path = os.path.join(directory, xml_files[number]) if number in xml_files else 'no xml'
        png_path = os.path.join(directory, png_files[number]) if number in png_files else 'no png'
        matched_files.append((xml_path, png_path))

    return matched_files

## 3. 각 페이지로부터 이미지와 XML 매칭

In [ ]:
# 사용 예시: 매칭된 파일 가져오기
directory = r'/content'
files = get_matched_files(directory)

In [ ]:
files

[('/content/page_01.xml', '/content/page_1.png'),
 ('/content/page_02.xml', '/content/page_2.png'),
 ('/content/page_03.xml', '/content/page_3.png'),
 ('/content/page_04.xml', '/content/page_4.png'),
 ('/content/page_05.xml', '/content/page_5.png'),
 ('/content/page_06.xml', '/content/page_6.png'),
 ('/content/page_07.xml', '/content/page_7.png')]

In [ ]:
system_prompt = '''당신이 해석할 이미지는 HTP 반응특성 정리입니다.
1. 중요한 내용이므로 요약하지말고 문법에 신경쓰면서 보이는 그대로 작성해주세요.
2. 내용을 임의로 바꾸지 마세요. 그리고 보이는 모든 내용을 다 적으십시오.
3. 단, 테이블은 풀어서 평문 또는 나열식으로 작성해주세요. 이미지에 없는 말은 적지마세요.
4. 테이블 풀어서 평문 또는 나열식으로 작성할 때 다른 행과 열이랑 헷갈리지 않게 값마다 잘 구분해서 적어주세요.
5. 테이블 해석할 때 통합셀들이 존재하니 구조를 잘 해석해서 작성해주시기 바랍니다. 어떤 게 어떤 것의 하위 내용인지를 명확히 하십시오
6. 당신의 의견은 궁금하지 않습니다. 해드렸습니다. 완성했습니다. 이런 표현도 적지마십시오. 이미지에 있는 내용만 적으십시오.
7. 만약 다단으로 구성되어져 있다면 좌측 테이블부터 먼저 작성하고 우측 테이블을 작성하십시오.
8. 당신에게 당신이 해석할 파일을 xml로 변경한 내용도 드리겠습니다. 페이지 해석할 때 참고하세요.
9. xml에 있는 텍스트는 반드시 해당 페이지에 존재한다는 겁니다. xml에 있는 텍스트를 빠트리지 마십시오.

자 당신이 헷갈리지 않도록 xml도 드렸습니다. 이미지를 더 잘 해석할 거라 믿습니다.
'''

In [ ]:
%%time
# 이미지와 XML 파일을 사용하여 AI 모델에 요청을 보내는 코드
result_lst = []
for file in tqdm(files):
    image_path = file[1]

    # 이미지를 base64 형식으로 인코딩
    base64_image = encode_image(image_path)

    if file[0] == 'no xml':
        print(file[1], '은 xml이 없습니다.')
        prompt = no_xml_system_prompt
    else:
        with open(file[0], 'r', encoding='utf-8') as f:
            xml_content = f.read()
        prompt = system_prompt + xml_content + '\n시작!'

    response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": prompt},
            {
                "type": "input_image",
                "image_url": f"data:image/jpeg;base64,{base64_image}",
            },
        ],
    }],
    )
    result_lst.append(response.output_text)

100%|██████████| 7/7 [01:35<00:00, 13.59s/it]

CPU times: user 150 ms, sys: 16 ms, total: 166 ms
Wall time: 1min 35s


In [ ]:
# 결과를 페이지와 연결하여 저장
result = []
for f, r in zip(files, result_lst):
    result.append({'content': r, 'source': 'page_' + f[1].split('page_')[1]})

In [ ]:
result

[{'content': '부록3. 성격 5요인에 따른 HTP 반응특성 정리\n\n<집 그림의 분석결과 정리>\n\n1. 항목: 집 크기  \n   상태: 크게 그리는 경우  \n   가능한 분석: 피험자가 상상력이 풍부하거나 책임감이 강할수록 크게 그리는 경향이 있다.\n\n2. 항목: 위치  \n   상태: 중앙에 그리는 경우  \n   가능한 분석: 피험자가 원만하며 사회성이 좋을수록 중앙에 그리는 경향이 있다.\n\n3. 항목: 필압  \n   상태: 보통  \n   가능한 분석: 보통의 필압으로 그리는 피험자의 경우, 민감하고 감각적인 경향이 있다.\n\n4. 항목: 선의 성질  \n   상태: 파선  \n   가능한 분석: 피험자가 자존감이 낮을수록 파선으로 그리는 경향이 있다.  \n   상태: 보통선  \n   가능한 분석: 보통의 선으로 그리는 피험자는 생각이 유연하고 책임감이 강한 경향이 있다.  \n   상태: 강한 직선  \n   가능한 분석: 주의가 산만하고 충동적일수록 강한 직선을 많이 사용하는 경향이 있다.\n\n5. 항목: 집의 형태  \n   상태: 양옥집  \n   가능한 분석: 양옥집을 그리는 피험자의 경우 타인의 말에 잘 공감해주는 경향이 있다.  \n   상태: 아파트  \n   가능한 분석: 아파트를 그리는 피험자의 경우 배려 깊고 관용적인 경향이 있다.\n\n6. 항목: 지붕 형태  \n   상태: 1차원  \n   가능한 분석: 피험자가 배려 깊고 관용적일수록, 상상력이 풍부하고 창의적일수록 1차원적인 지붕을 그리는 경향이 있다.  \n   상태: 3차원  \n   가능한 분석: 피험자가 수용적이고 신뢰를 잘 할수록, 생각이 다양하고 상상력이 풍부할수록 3차원적인 지붕을 그리는 경향이 있다.\n\n7. 항목: 굴뚝  \n   상태: 없다  \n   가능한 분석: 피험자가 창의적이거나 호기심이 많을수록, 야심이 있고 목표지향적일수록 굴뚝을 그리지 않는 경향이 있다.\n\n8. 항목: 창문 종류  \n   상태: 가려진 창

In [ ]:
for sample in result:
  print(sample)
  print('==' * 50)

{'content': '부록3. 성격 5요인에 따른 HTP 반응특성 정리\n\n<집 그림의 분석결과 정리>\n\n1. 항목: 집 크기  \n   상태: 크게 그리는 경우  \n   가능한 분석: 피험자가 상상력이 풍부하거나 책임감이 강할수록 크게 그리는 경향이 있다.\n\n2. 항목: 위치  \n   상태: 중앙에 그리는 경우  \n   가능한 분석: 피험자가 원만하며 사회성이 좋을수록 중앙에 그리는 경향이 있다.\n\n3. 항목: 필압  \n   상태: 보통  \n   가능한 분석: 보통의 필압으로 그리는 피험자의 경우, 민감하고 감각적인 경향이 있다.\n\n4. 항목: 선의 성질  \n   상태: 파선  \n   가능한 분석: 피험자가 자존감이 낮을수록 파선으로 그리는 경향이 있다.  \n   상태: 보통선  \n   가능한 분석: 보통의 선으로 그리는 피험자는 생각이 유연하고 책임감이 강한 경향이 있다.  \n   상태: 강한 직선  \n   가능한 분석: 주의가 산만하고 충동적일수록 강한 직선을 많이 사용하는 경향이 있다.\n\n5. 항목: 집의 형태  \n   상태: 양옥집  \n   가능한 분석: 양옥집을 그리는 피험자의 경우 타인의 말에 잘 공감해주는 경향이 있다.  \n   상태: 아파트  \n   가능한 분석: 아파트를 그리는 피험자의 경우 배려 깊고 관용적인 경향이 있다.\n\n6. 항목: 지붕 형태  \n   상태: 1차원  \n   가능한 분석: 피험자가 배려 깊고 관용적일수록, 상상력이 풍부하고 창의적일수록 1차원적인 지붕을 그리는 경향이 있다.  \n   상태: 3차원  \n   가능한 분석: 피험자가 수용적이고 신뢰를 잘 할수록, 생각이 다양하고 상상력이 풍부할수록 3차원적인 지붕을 그리는 경향이 있다.\n\n7. 항목: 굴뚝  \n   상태: 없다  \n   가능한 분석: 피험자가 창의적이거나 호기심이 많을수록, 야심이 있고 목표지향적일수록 굴뚝을 그리지 않는 경향이 있다.\n\n8. 항목: 창문 종류  \n   상태: 가려진 창문

In [ ]:
import json

# result 리스트 저장
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print("✅ result.json 파일로 저장 완료!")


✅ result.json 파일로 저장 완료!


In [ ]:
#result.json 파일 불러오기
with open('result.json', 'r', encoding='utf-8') as f:
    loaded_result = json.load(f)

print(loaded_result[0])

{'content': '부록3. 성격 5요인에 따른 HTP 반응특성 정리\n\n<집 그림의 분석결과 정리>\n\n1. 항목: 집 크기  \n   상태: 크게 그리는 경우  \n   가능한 분석: 피험자가 상상력이 풍부하거나 책임감이 강할수록 크게 그리는 경향이 있다.\n\n2. 항목: 위치  \n   상태: 중앙에 그리는 경우  \n   가능한 분석: 피험자가 원만하며 사회성이 좋을수록 중앙에 그리는 경향이 있다.\n\n3. 항목: 필압  \n   상태: 보통  \n   가능한 분석: 보통의 필압으로 그리는 피험자의 경우, 민감하고 감각적인 경향이 있다.\n\n4. 항목: 선의 성질  \n   상태: 파선  \n   가능한 분석: 피험자가 자존감이 낮을수록 파선으로 그리는 경향이 있다.  \n   상태: 보통선  \n   가능한 분석: 보통의 선으로 그리는 피험자는 생각이 유연하고 책임감이 강한 경향이 있다.  \n   상태: 강한 직선  \n   가능한 분석: 주의가 산만하고 충동적일수록 강한 직선을 많이 사용하는 경향이 있다.\n\n5. 항목: 집의 형태  \n   상태: 양옥집  \n   가능한 분석: 양옥집을 그리는 피험자의 경우 타인의 말에 잘 공감해주는 경향이 있다.  \n   상태: 아파트  \n   가능한 분석: 아파트를 그리는 피험자의 경우 배려 깊고 관용적인 경향이 있다.\n\n6. 항목: 지붕 형태  \n   상태: 1차원  \n   가능한 분석: 피험자가 배려 깊고 관용적일수록, 상상력이 풍부하고 창의적일수록 1차원적인 지붕을 그리는 경향이 있다.  \n   상태: 3차원  \n   가능한 분석: 피험자가 수용적이고 신뢰를 잘 할수록, 생각이 다양하고 상상력이 풍부할수록 3차원적인 지붕을 그리는 경향이 있다.\n\n7. 항목: 굴뚝  \n   상태: 없다  \n   가능한 분석: 피험자가 창의적이거나 호기심이 많을수록, 야심이 있고 목표지향적일수록 굴뚝을 그리지 않는 경향이 있다.\n\n8. 항목: 창문 종류  \n   상태: 가려진 창문

In [ ]:
#result.json 파일 불러오기
with open('result.json', 'r', encoding='utf-8') as f:
    loaded_result = json.load(f)

print(loaded_result[0])

In [ ]:
# result 리스트를 txt 파일로 저장
with open('result.txt', 'w', encoding='utf-8') as f:
    for item in result:
        f.write(str(item) + "\n\n")  # 각 아이템 구분을 위해 줄바꿈

print("✅ result.txt 파일로 저장 완료!")


✅ result.txt 파일로 저장 완료!


# RAG 실습

In [ ]:
!pip install langchain langchain_openai langchain_community chromadb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain import PromptTemplate

In [ ]:
# 문서들을 전부 문자열로 변환
docs = [str(r) for r in result]

print(type(docs[0]))
print('0번 문서 출력:', docs[0])

In [ ]:
print(docs)

In [ ]:
# 랭체인 문서 포맷으로 변환
langchain_format_docs = []

for doc in docs:
  langchain_format_docs.append(Document(page_content=doc))

print(type(langchain_format_docs[0]))
print('0번 문서 출력:', langchain_format_docs[0])

In [ ]:
langchain_format_docs

In [ ]:
import re
from langchain.schema import Document

final_docs = []

for doc in langchain_format_docs:
    text = doc.page_content

    # 큰 카테고리 추출: 예를 들어 '<집 그림 분석 결과 정리>'
    # 패턴: <...> 형식
    main_category_match = re.search(r'<([^>]+)>', text)
    main_category = main_category_match.group(1) if main_category_match else "기타"

    # 1. 큰 번호 기준으로 split
    chunks = re.split(r'(?=\n?\d+\.)', text)

    for chunk in chunks:
        # 2. 항목 기준으로 split
        sub_chunks = re.split(r'(?=\n\s*-?\s*항목:)', chunk)
        for sub_chunk in sub_chunks:
            sub_chunk = sub_chunk.strip()
            if not sub_chunk:
                continue

            # 3. 번호/제목 추출
            match = re.match(r'(\d+)\.\s*항목:\s*(.+)', sub_chunk)
            if match:
                number = match.group(1)
                title = match.group(2).strip()
            else:
                number = None
                title = sub_chunk[:20] + "..."  # 항목 없으면 일부 텍스트

            # 4. metadata에 main_category 포함
            final_docs.append(
                Document(
                    page_content=sub_chunk,
                    metadata={
                        "category": f"{main_category} - {number + ' ' if number else ''}{title}"
                    }
                )
            )

# 결과 확인 (상위 5개)
for doc in final_docs:
    print(doc.page_content[:200])
    print('-'*50)


In [ ]:
final_docs

In [ ]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    collection_name="your_collection_name",
    documents=final_docs,
    embedding=embedding)

In [ ]:
# 벡터DB의 개수 확인
vectordb._collection.count()

In [ ]:
for key in vectordb._collection.get():
  print(key)

In [ ]:
# 문서 로드
documents = vectordb._collection.get()['documents']
print('문서의 개수 :', len(documents))
print('-' * 100)
print('첫번째 문서 출력 :', documents[0])

In [ ]:
# 유사도가 높은 문서 5개만 추출. k = 5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

top_5_docs = retriever.get_relevant_documents("큰 나무 그림")
print('유사 문서 개수 :', len(top_5_docs))
print('--' * 20)
for doc in top_5_docs:
  print(doc)
  print('--')

In [ ]:
template = """당신은 심리를 해석해주는 챗봇입니다.

주어진 검색 결과를 바탕으로 답변하세요.

1. 당신은 오직 위에서 제공된 참고 자료에 있는 사실 정보에만 근거해 사용자 질문에 답변해야 하며, 절대 지어내거나 허구의 정보를 포함해서는 안 됩니다.
2. 만약 사용자의 질문을 명확히 하는 것이 답변에 도움이 된다면, 질문을 시도할 수 있습니다.
3. 만약 참고 자료에 있는 정보로 질문에 충분히 답변할 수 없다면, 다음의 문장을 그대로 답변으로 사용해야 합니다: "죄송합니다. 참고 자료에는 요청하신 질문에 답변할 만한 충분한 정보가 없네요.".
4. 만약 답변에 참고 자료의 내용을 인용했다면, 답변에 사용한 문장이나 문단 끝마다 해당 참고 자료의 Source_id를 반드시 추가해야 합니다. Source_id 값은 참고 자료에서 가져오며, 두 개의 대괄호로 감싸야 합니다. 예시: [[page_21.png]], [[page_29.png]]
5. 반드시 한국어로 1인칭 시점에서 정확하고 엄격한 스타일로 답변해야 하며, 사실을 기반으로 상세히 설명해야 합니다.

{context}

Question: {question}
Answer:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,
    return_source_documents=True)

In [ ]:
def get_chatbot_response(input_text):
    chatbot_response = qa_chain.invoke(input_text)
    return chatbot_response['result'].strip()

In [ ]:
input_text = "자신감이 부족하면 나무의 그림이 어떤 형태야?"
result = get_chatbot_response(input_text)
print(result)